In [1]:
!pip install fitz

In [2]:
!pip install PyMuPDF

In [3]:
import fitz  # PyMuPDF

def extract_text_from_pdf(file_path):
    doc = fitz.open(file_path)
    text = ""
    for page_num, page in enumerate(doc, start=1):
        text += page.get_text() + "\n"
    return text

# Danh sách các file PDF
pdf_files = [
    "Toi-thay-hoa-vang-tren-co-xanh.pdf",
    "2363-bay-buoc-toi-mua-he-thuviensach.vn.pdf",
    "Co 2 con meo ngoi ben cua so.pdf",
    "mat-biec-nguyen-nhat-anh_146202310.pdf",
    "toi-la-beto-nguyen-nhat-anh-thuviensachvn_82202392236.pdf"
]

# Nối toàn bộ nội dung thành 1 biến
all_text = ""
for pdf in pdf_files:
    all_text += extract_text_from_pdf(pdf)

# In hoặc lưu kết quả


In [4]:
paragraphs = [p.strip() for p in all_text.split('\n\n') if p.strip()]

In [5]:
for i, s in enumerate(paragraphs[:10], 1):
    print(f"{i}: {s}")

1: Nguyễn Nhật Ánh
TÔI THẤY HOA VÀNG
TRÊN CỎ XANH
Thư viện online
isach.info
2: Thông tin về ebook
Tôi Thấy Hoa Vàng Trên Cỏ Xanh
Tác giả: Nguyễn Nhật Ánh
Thể loại: Truyện Ngắn
Biên tập: Little Rain
Bìa: truonghoangngan
Định dạng ebook PDF-A4
Ngày xuất bản: 10-September-2015
Tổng số 227 trang
Thư viện online isach.info
3: CHƯƠNG 1
 
HOA TAY
 
 Chú Đàn bảo tôi:
  
- Con xòe tay ra cho chú xem nào!
  
Tôi co những ngón tay lại, nắm thật chặt và giấu ra sau lưng:
  
- Tay con sạch cơ mà. Hồi sáng con đã rửa tay rồi.
  
Chú Đàn phì cười:
  
- Chú có định khám tay con đâu. Con xòe tay ra đểchú xem con có mấy
cái hoa tay thôi.
  
Đằng sau lưng, hai bàn tay tôi lỏng đi. Tôi chìa bàn tay trái ra trước mặt
chú Đàn, thắc mắc:
  
- Hoa tay là gì hở chú?
  
Chú Đàn dựng mắt nhìn tôi:
  
- Con lớn từng này rồi mà không biết hoa tay là gì à?
  
Chú cầm lấy bàn tay tôi, chậm rãi giải thích:
  
- Hoa tay là những vân tay hình tròn ởđầu mỗi ngón tay. Hoa tay càng
nhiều thì vẽcàng đẹp. Nếu con có mười c

In [6]:
corpus = paragraphs
model_name = "meta-llama/Llama-3.2-1B"

In [7]:
!pip install datasets

In [8]:
def group_sentences(data, group_size=4, stride=2):
    """
    Nhóm các câu thành các mẫu chồng lên nhau có kích thước xác định với khoảng cách chỉ định.

    Tham số:
        data (list of str): Danh sách các câu.
        group_size (int): Số lượng câu trong mỗi nhóm.
        stride (int): Bước nhảy cho các nhóm.

    Kết quả trả về:
        list of str: Danh sách các câu đã được nhóm, nối bằng ký tự xuống dòng.
    """
    grouped_data = []
    for i in range( 0 , len(data) , stride):
      if i + group_size <= len(data):
        tmp  = "\n".join(data[i:i+ group_size]) + "\n"
        grouped_data.append(tmp)
    return grouped_data


In [9]:
from datasets import Dataset, DatasetDict

# Group sentences into overlapping samples of 4 with a stride of 2
grouped_sentences = group_sentences(corpus, 4, 2)

# Split into train and validation datasets
split_index = int(0.8 * len(grouped_sentences))
dataset = DatasetDict({
    "train": Dataset.from_dict({"text": grouped_sentences[:split_index]}),
    "validation": Dataset.from_dict({"text": grouped_sentences[split_index:]})
})


In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 355
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 89
    })
})

In [11]:
print(dataset['train'][0]['text'])

Nguyễn Nhật Ánh
TÔI THẤY HOA VÀNG
TRÊN CỎ XANH
Thư viện online
isach.info
Thông tin về ebook
Tôi Thấy Hoa Vàng Trên Cỏ Xanh
Tác giả: Nguyễn Nhật Ánh
Thể loại: Truyện Ngắn
Biên tập: Little Rain
Bìa: truonghoangngan
Định dạng ebook PDF-A4
Ngày xuất bản: 10-September-2015
Tổng số 227 trang
Thư viện online isach.info
CHƯƠNG 1
 
HOA TAY
 
 Chú Đàn bảo tôi:
  
- Con xòe tay ra cho chú xem nào!
  
Tôi co những ngón tay lại, nắm thật chặt và giấu ra sau lưng:
  
- Tay con sạch cơ mà. Hồi sáng con đã rửa tay rồi.
  
Chú Đàn phì cười:
  
- Chú có định khám tay con đâu. Con xòe tay ra đểchú xem con có mấy
cái hoa tay thôi.
  
Đằng sau lưng, hai bàn tay tôi lỏng đi. Tôi chìa bàn tay trái ra trước mặt
chú Đàn, thắc mắc:
  
- Hoa tay là gì hở chú?
  
Chú Đàn dựng mắt nhìn tôi:
  
- Con lớn từng này rồi mà không biết hoa tay là gì à?
  
Chú cầm lấy bàn tay tôi, chậm rãi giải thích:
  
- Hoa tay là những vân tay hình tròn ởđầu mỗi ngón tay. Hoa tay càng
nhiều thì vẽcàng đẹp. Nếu con có mười cái hoa ta

In [12]:
grouped_sentences[0]

'Nguyễn Nhật Ánh\nTÔI THẤY HOA VÀNG\nTRÊN CỎ XANH\nThư viện online\nisach.info\nThông tin về ebook\nTôi Thấy Hoa Vàng Trên Cỏ Xanh\nTác giả: Nguyễn Nhật Ánh\nThể loại: Truyện Ngắn\nBiên tập: Little Rain\nBìa: truonghoangngan\nĐịnh dạng ebook PDF-A4\nNgày xuất bản: 10-September-2015\nTổng số 227 trang\nThư viện online isach.info\nCHƯƠNG 1\n \nHOA TAY\n \n Chú Đàn bảo tôi:\n  \n- Con xòe tay ra cho chú xem nào!\n  \nTôi co những ngón tay lại, nắm thật chặt và giấu ra sau lưng:\n  \n- Tay con sạch cơ mà. Hồi sáng con đã rửa tay rồi.\n  \nChú Đàn phì cười:\n  \n- Chú có định khám tay con đâu. Con xòe tay ra đểchú xem con có mấy\ncái hoa tay thôi.\n  \nĐằng sau lưng, hai bàn tay tôi lỏng đi. Tôi chìa bàn tay trái ra trước mặt\nchú Đàn, thắc mắc:\n  \n- Hoa tay là gì hở chú?\n  \nChú Đàn dựng mắt nhìn tôi:\n  \n- Con lớn từng này rồi mà không biết hoa tay là gì à?\n  \nChú cầm lấy bàn tay tôi, chậm rãi giải thích:\n  \n- Hoa tay là những vân tay hình tròn ởđầu mỗi ngón tay. Hoa tay càng\nnhi

In [13]:
grouped_sentences[1]

'CHƯƠNG 1\n \nHOA TAY\n \n Chú Đàn bảo tôi:\n  \n- Con xòe tay ra cho chú xem nào!\n  \nTôi co những ngón tay lại, nắm thật chặt và giấu ra sau lưng:\n  \n- Tay con sạch cơ mà. Hồi sáng con đã rửa tay rồi.\n  \nChú Đàn phì cười:\n  \n- Chú có định khám tay con đâu. Con xòe tay ra đểchú xem con có mấy\ncái hoa tay thôi.\n  \nĐằng sau lưng, hai bàn tay tôi lỏng đi. Tôi chìa bàn tay trái ra trước mặt\nchú Đàn, thắc mắc:\n  \n- Hoa tay là gì hở chú?\n  \nChú Đàn dựng mắt nhìn tôi:\n  \n- Con lớn từng này rồi mà không biết hoa tay là gì à?\n  \nChú cầm lấy bàn tay tôi, chậm rãi giải thích:\n  \n- Hoa tay là những vân tay hình tròn ởđầu mỗi ngón tay. Hoa tay càng\nnhiều thì vẽcàng đẹp. Nếu con có mười cái hoa tay, con sẽvẽđẹp nhất\nlớp. Con viết chữ cũng đẹp nhất lớp.\n  \nTôi hồi hộp nhìn chú Đàn săm soi từng ngón tay tôi. Có cảm giác như\nchú đang nghiên cứu những chiếc gân lá trên năm chiếc lá.\n  \nTôi nín thở, hỏi:\n  \n- Con có mấy cái hoa tay hả chú?\n  \nChú Đàn lắc đầu, thất vọng:\n

In [14]:
import random

In [15]:
from huggingface_hub import login
from google.colab import userdata
# Authenticate with your Hugging Face token
login(
    userdata.get("hugging_face_token")
)

In [16]:
from transformers import AutoTokenizer
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Use EOS as PAD token

# Tokenization
def tokenize_function(examples):
    """
    Hàm này thực hiện việc mã hóa (tokenize) dữ liệu văn bản để chuẩn bị đưa vào mô hình ngôn ngữ.

    Tham số:
        examples (dict): Một từ điển chứa các cặp khóa-giá trị,
                         trong đó khóa 'text' chứa danh sách các chuỗi văn bản cần mã hóa.

    Trả về:
        dict: Một từ điển chứa các thành phần sau:
            - input_ids: Danh sách ID của các token đã được mã hóa từ văn bản.
            - attention_mask: Mặt nạ chú ý, giúp mô hình bỏ qua các giá trị đệm (padding).
            - labels: Sao chép từ `input_ids` để sử dụng làm nhãn trong các bài toán huấn luyện có giám sát.
    Chú ý:
        - Tham số `max_length` nên được điều chỉnh phù hợp với kích thước tối đa của mô hình.
    """

    # Mã hóa văn bản
    # - Cắt bớt văn bản nếu nó vượt quá `max_length`.
    # - Thêm giá trị đệm (padding) vào văn bản ngắn hơn `max_length`.
    # - Giới hạn tối đa độ dài chuỗi mã hóa là 64 token.
    tokens = tokenizer(examples["text"], truncation=True, padding='max_length', max_length=512)

    # Sao chép từ `input_ids` để sử dụng làm nhãn trong các bài toán huấn luyện có giám sát.
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens


tokenized_datasets = dataset.map(tokenize_function, batched=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/355 [00:00<?, ? examples/s]

Map:   0%|          | 0/89 [00:00<?, ? examples/s]

In [17]:
tokenized_datasets["train"][0]["input_ids"]

[128000,
 21567,
 105166,
 108211,
 43912,
 17134,
 198,
 51,
 106865,
 40,
 4534,
 121500,
 56,
 31350,
 32,
 650,
 73053,
 6269,
 198,
 2434,
 107267,
 45,
 356,
 5080,
 236,
 1630,
 1111,
 39,
 198,
 1016,
 19470,
 110847,
 2930,
 198,
 285,
 613,
 5506,
 198,
 99141,
 25826,
 100690,
 35097,
 198,
 127806,
 666,
 85555,
 107559,
 650,
 52649,
 1183,
 28119,
 356,
 86242,
 1630,
 29024,
 198,
 51,
 30441,
 107951,
 25,
 105947,
 108211,
 43912,
 17134,
 198,
 1016,
 34550,
 101278,
 25,
 1183,
 102209,
 17030,
 102675,
 198,
 37196,
 28119,
 102455,
 25,
 15013,
 22674,
 198,
 33,
 24439,
 64,
 25,
 490,
 69895,
 6292,
 526,
 983,
 276,
 198,
 41325,
 109288,
 105765,
 35097,
 11612,
 6830,
 19,
 198,
 119733,
 101616,
 101764,
 25,
 220,
 605,
 12,
 30649,
 12,
 679,
 20,
 198,
 51,
 103517,
 66626,
 220,
 14206,
 103763,
 198,
 1016,
 19470,
 110847,
 2930,
 374,
 613,
 5506,
 198,
 2198,
 110583,
 123097,
 6269,
 220,
 16,
 27907,
 24755,
 32,
 350,
 3097,
 27907,
 921,
 6792,
 5

In [18]:
sample_ids = tokenize_function(dataset["train"][0])

In [19]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 355
    })
    validation: Dataset({
        features: ['text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 89
    })
})

In [20]:
tokens = [tokenizer.decode([token_id]) for token_id in tokenized_datasets["train"]["input_ids"][0]]

print(tokens)

['<|begin_of_text|>', 'Ng', 'uyễn', ' Nhật', ' Á', 'nh', '\n', 'T', 'Ô', 'I', ' TH', 'Ấ', 'Y', ' HO', 'A', ' V', 'À', 'NG', '\n', 'TR', 'Ê', 'N', ' C', '�', '�', ' X', 'AN', 'H', '\n', 'Th', 'ư', ' viện', ' online', '\n', 'is', 'ach', '.info', '\n', 'Thông', ' tin', ' về', ' ebook', '\n', 'Tôi', ' Th', 'ấy', ' Hoa', ' V', 'àng', ' Tr', 'ên', ' C', 'ỏ', ' X', 'anh', '\n', 'T', 'ác', ' giả', ':', ' Nguyễn', ' Nhật', ' Á', 'nh', '\n', 'Th', 'ể', ' loại', ':', ' Tr', 'uyện', ' Ng', 'ắn', '\n', 'Bi', 'ên', ' tập', ':', ' Little', ' Rain', '\n', 'B', 'ì', 'a', ':', ' tr', 'uong', 'ho', 'ang', 'ng', 'an', '\n', 'Đ', 'ịnh', ' dạng', ' ebook', ' PDF', '-A', '4', '\n', 'Ngày', ' xuất', ' bản', ':', ' ', '10', '-', 'September', '-', '201', '5', '\n', 'T', 'ổng', ' số', ' ', '227', ' trang', '\n', 'Th', 'ư', ' viện', ' online', ' is', 'ach', '.info', '\n', 'CH', 'Ư', 'Ơ', 'NG', ' ', '1', '\n \n', 'HO', 'A', ' T', 'AY', '\n \n', ' Ch', 'ú', ' Đ', 'àn', ' bảo', ' tôi', ':\n', '  \n', '-', ' Con', ' 

In [21]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Causal Language Modeling
)

In [22]:
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments
import torch

# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16
)


In [23]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./llama-pretrained-nguyen-nhat-anh",  # Thư mục để lưu các điểm kiểm tra (checkpoint) của mô hình
    eval_strategy="epoch",  # Đánh giá mô hình sau mỗi epoch (1 vòng lặp qua toàn bộ dữ liệu huấn luyện)
    save_strategy="epoch",  # Lưu checkpoint của mô hình sau mỗi epoch
    load_best_model_at_end=True,  # Tải mô hình tốt nhất dựa trên tổn thất (loss) khi đánh giá
    metric_for_best_model="eval_loss",  # Sử dụng tổn thất đánh giá (eval loss) để xác định mô hình tốt nhất
    greater_is_better=False,  # Tổn thất càng nhỏ càng tốt (loss thấp hơn là tốt hơn)
    learning_rate=2e-5,  # Tốc độ học (learning rate) của trình tối ưu hóa
    per_device_train_batch_size=2,  # Kích thước batch cho mỗi thiết bị khi huấn luyện
    per_device_eval_batch_size=2,  # Kích thước batch cho mỗi thiết bị khi đánh giá
    num_train_epochs=10,  # Số lượng epoch huấn luyện (số vòng lặp qua toàn bộ tập dữ liệu huấn luyện)
    weight_decay=0.01,  # Hệ số suy giảm trọng số để tránh quá khớp (overfitting)
    logging_dir="./logs",  # Thư mục lưu trữ log để theo dõi quá trình huấn luyện
    logging_steps=10,  # Ghi log sau mỗi 10 bước huấn luyện
    save_total_limit=2  # Chỉ giữ lại 2 checkpoint gần nhất để tiết kiệm dung lượng lưu trữ
)


In [24]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Start Training
trainer.train()

<ipython-input-24-821de0b18636>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: teamaiproptit (teamaiproptit-other) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,2.921700,2.875080
2,2.464400,2.881242
3,2.086600,3.011683
4,1.642400,3.141877
5,1.499800,3.347493
6,1.235400,3.505677
7,1.088400,3.631246
8,0.938900,3.740740
9,0.860200,3.785973
10,0.905800,3.801198


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=1780, training_loss=1.5684955771049756, metrics={'train_runtime': 8541.0901, 'train_samples_per_second': 0.416, 'train_steps_per_second': 0.208, 'total_flos': 1.06127422390272e+16, 'train_loss': 1.5684955771049756, 'epoch': 10.0})

In [25]:
# Evaluate the model
eval_results = trainer.evaluate()

# Print evaluation results
print("Evaluation Results:")
for key, value in eval_results.items():
    print(f"{key}: {value}")


Evaluation Results:
eval_loss: 2.8750803470611572
eval_runtime: 51.233
eval_samples_per_second: 1.737
eval_steps_per_second: 0.878
epoch: 10.0


In [26]:
import torch
from transformers import AutoModelForCausalLM

# Load the best model
best_model_path = trainer.state.best_model_checkpoint
print(f"Best model path: {best_model_path}")

best_model = AutoModelForCausalLM.from_pretrained(
    best_model_path
).to("cuda" if torch.cuda.is_available() else "cpu")

Best model path: ./llama-pretrained-nguyen-nhat-anh/checkpoint-178


In [27]:
# Generate text using the best model
input_text = "Nếu em"
inputs = tokenizer(input_text, return_tensors="pt").to(best_model.device)

with torch.no_grad():
    outputs = best_model.generate(
        **inputs,
        max_length=300,
        num_return_sequences=1,
        temperature=0.7,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated text from the best model:")
print(generated_text)


Generated text from the best model:
Nếu em không biết cách chơi bây giờ, em sẽ nói cho em biết. Em có
được một trò chơi không?
- Một trò chơi gì đó? - Em nhếch môi - Tao không biết trò chơi gì đó
nhưng em có một trò chơi rất thú vị.
- Hỏi em. Em có một trò chơi gì đó không? - Tối hôm đó, em hỏi
bây giờ, và em thấy tôi ngoảnh đầu lại.
- Hỏi em. - Tôi đáp, và tôi mở miệng ra, giọng trầm tư - Em có trò chơi gì
không?
- Em không biết trò chơi gì đó. - Em cười - Em sẽ hỏi em về trò chơi đó.
Em hỏi về trò chơi đó. - Em không biết trò chơi gì đó. - Em ngoảnh đầu lại
như muốn hỏi thêm. - Em có trò chơi gì không? - Em hỏi lần nữa, giọng
trầm tư.
- Em không biết trò chơi gì đó. - Em nhếch môi, giọng trầm tư. - Em có trò
chơi gì không? - Em hỏi lần nữa, giọng trầm tư.
Tôi ngoảnh đầu lại. Tôi không biết trò chơi gì đó. - Em hỏi, giọng trầm
trú. - Em có trò chơi gì không? - Em hỏi lần nữa, giọng trầm tư.
- Em không biết trò chơi gì


In [28]:
trainer.push_to_hub(commit_message = "Training completed!")


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Dat1710/llama-pretrained-nguyen-nhat-anh/commit/b00efddd9d8ade633ddf7eb0cb99ba85717a90af', commit_message='Training completed!', commit_description='', oid='b00efddd9d8ade633ddf7eb0cb99ba85717a90af', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Dat1710/llama-pretrained-nguyen-nhat-anh', endpoint='https://huggingface.co', repo_type='model', repo_id='Dat1710/llama-pretrained-nguyen-nhat-anh'), pr_revision=None, pr_num=None)

In [1]:
from transformers import pipeline
# Change `Dat1710` to your Hub username
model_id = "Dat1710/llama-pretrained-nguyen-nhat-anh"
classifier = pipeline("text2text-generation", model=model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0
The model 'LlamaForCausalLM' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTMod

In [15]:
text = "Cậu không biết khi nào tình yêu sẽ đến, nhưng cậu sẽ biết khi nào nó thực sự ra đi."
preds = classifier(
    text,
    max_length=200,               # Giới hạn độ dài đầu ra
    num_return_sequences=3,       # Trả về 3 kết quả khác nhau
    do_sample=True,               # Bật sampling để kết quả không lặp
    temperature=0.7,              # Điều chỉnh độ ngẫu nhiên (0.7–1.0 là hợp lý)
)

for i, pred in enumerate(preds):
    print(f"[Kết quả {i+1}]")
    print(pred['generated_text'])

[Kết quả 1]
Cậu không biết khi nào tình yêu sẽ đến, nhưng cậu sẽ biết khi nào nó thực sự ra đi. Cậu sẽ biết khi nó đi, đi đi đi, đi đi đi, đi đi đi.
Cậu sẽ biết khi nó đi, đi đi đi, đi đi đi, đi đi đi.
- Đợi ba về . - Cậu nhủ bụng, không biết có đúng không, nhưng cậu có cảm giác như một
chú mèo đang ngậm chiếc nhúm trên đầu của chú.
- Ba về. - Cậu nhủ bụng, không biết có đúng không, nhưng cậu có cảm giác như một
chú mèo đang ngậm chiếc nhúm trên đầu của chú.
- Ba về, ba về, ba về, ba về... - Cậu cười ứa nước mắt, lần thứ hai trong một
nữa.
Cậu bỗng thấy cơn sốt ruột bùng nổ. Cậu nhìn
[Kết quả 2]
Cậu không biết khi nào tình yêu sẽ đến, nhưng cậu sẽ biết khi nào nó thực sự ra đi. Cậu sẽ biết khi nào nó không còn ở bên cạnh cậu nữa. Cậu sẽ biết khi nào nó không còn là một mẩu xương rời bỏ đi. Cậu sẽ biết khi nào nó không còn là một dấu hỏi trong đầu cậu nữa.
Hôm nay, cậu không biết khi nào tình yêu sẽ đến, nhưng chắc chắn đó sẽ là một ngày tuyệt vời. Ngày đó sẽ rất ngọt ngào, rất ấm áp và